# KMeans Clustering - Deployment

This is a component that trains a KMeans model using [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use SDK to load the dataset and save a model.
- how to receive parameters from the platform.

## Wrapping Model

Allows your component to expose a service over REST.

To wrap your model [follow the instructions](https://docs.seldon.io/projects/seldon-core/en/v0.3.0/python/python_component.html) for your chosen language or toolkit.

In [ ]:
%%writefile Model.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model

logger = logging.getLogger(__name__)


class Model(object):

    def __init__(self, dataset=None, target=None):
        
        logger.info(f"dataset: {dataset}")

        # Load Artifacts: Estimator, Encoders, etc
        model = load_model()
        self.pipeline = model["pipeline"]
        self.columns = model["columns"]
    
    def class_names(self):
        return ["cluster"]

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        """Takes an array (numpy) X and feature_names and returns an array of predictions.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.
            meta (dict, optional): Dict of metadata.
        """
        # Put data in a pandas.DataFrame to sort it
        df = pd.DataFrame(X, columns=feature_names)

        # Put data back in a numpy.ndarray
        X = df[self.columns].to_numpy()

        y_pred = self.pipeline.predict(X)

        return y_pred

## API Contract

There are two sections:

- `features` : The feature array you intend to send in a request

Each section has a list of definitions. Each definition consists of:

- `name` : String : The name of the feature
- `ftype` : one of CONTINUOUS, CATEGORICAL : the type of the feature
- `dtype` : One of FLOAT, INT : Required for ftype CONTINUOUS : What type of feature to create
- `values` : list of Strings : Required for ftype CATEGORICAL : The possible categorical values
- `range` : list of two numbers : Optional for ftype CONTINUOUS : The range of values (inclusive) that a continuous value can take
- `repeat` : integer : Optional value for how many times to repeat this value
- `shape` : array of integers : Optional value for the shape of array to coerce the values

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "SepalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [4.3, 7.9]
        },
        {
            "name": "SepalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [2.0, 4.4]
        },
        {
            "name": "PetalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 6.9]
        },
        {
            "name": "PetalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.1, 2.5]
        }
    ],
    "targets": [
        {
            "name": "cluster",
            "ftype": "categorical",
            "dtype": "INT",
            "values": [1, 2, 3]
        }
    ]
}

## Test Deployment

Starts a service wrapping a Model, sends a request to the service, and shows the response.

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")